
# Capstone Project 

# Author : Hamidreza Salahi

# Notebook : 1


# Table of Contents
* [Introduction](#Introduction)
* [Importing Data](#Importing-Data)
* [Creating Sample Data](#Creating-Sample-Data)
* [Data Cleaning](#Data-Cleaning)
    * [Dropping NaN Columns](#Dropping-NaN-Columns)
    * [Data Dictionary](#Data-Dictionary)
    * [Dropping NaN Rows](#Dropping-NaN-Rows)
    * [Converting Non-numeric Columns to Numeric Columns](#Converting-Non-numeric-Columns-to-Numeric-Columns)
* [Feature Selection](#Feature-Selection)    
* [References](#References)

# Introduction

LendingClub is one of the largest peer-to-peer financial services company headquartered in San Francisco, California. It was the first peer-to-peer lender to register its offerings as securities with the Securities and Exchange Commission (SEC), and to offer loan trading on a secondary market. The company reported that $15.98 billion in loans had been originated through its platform up to December 31, 2015 [1]. <br>

In this project, it is assumed that I am recruited in LendingClub company as Data Scientist to answer the following business question: <br>

**Using machine learning (ML), make a loan approval predictor that identifies whether an applicant is risky or not**. <br>

Here, a risky applicant is an applicant who will not be able to pay the instalments in due time for a long period of time. In the LendingClub dataset, `Charged off` term in the load_status column refers to those applicants who has not paid their loan and has defaulted on the loan  <br>

Having a ML indicator can be very helpful for the business for two reasons: <br>

(i) Approving a risky applicant may lead to financial loss <br>
(ii) Not approving an applicant who is likely to pay their loan also leads to loss of business profit. 


In this project, the target variable is `loan_status` column. Within this column, I will concentrate only on `Fully Paid` and `Charged off` applicants i.e., `Current`, `Late (16-30 days)`, `Late (31-120 days)`, `In Grace Period`, `Issued`, `Does not meet the credit policy. Status:Fully Paid`, `Does not meet the credit policy. Status:Charged Off` and  `Default` applicants will not be considered in the analysis. 

## Importing Data

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Creating dataframe from dataset zip file
loans_df = pd.read_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\Loan_status_2007-2020Q3.gzip', compression='gzip', low_memory=False, index_col=0)
loans = loans_df.copy()
loans.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,1076863,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,1075358,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


Starting by looking at the distribution of the target variable, `loan_status` column

In [3]:
loans['loan_status'].value_counts(normalize=True)

Fully Paid                                             0.511976
Current                                                0.352425
Charged Off                                            0.123927
Late (31-120 days)                                     0.005522
In Grace Period                                        0.003428
Late (16-30 days)                                      0.000929
Issued                                                 0.000705
Does not meet the credit policy. Status:Fully Paid     0.000680
Does not meet the credit policy. Status:Charged Off    0.000260
Default                                                0.000148
Name: loan_status, dtype: float64

Only the `Fully Paid` and `Charged Off` applicants will be retained and other categories will be dropped. Except for the `Current` applicants, which contains almost 35% of the applicants, the other ignored fields combined contribut less than 1.3%  of the dataset.  <br>
One issue here is that a significant imbalance in the population of `Fully Paid` and `Charged Off` applicants is seen. This issue is addressed in the next section below. 

## Creating Sample Data

Since the original data set is too large, we create a sample data set to work with for now. The sample data set contains  10% of the original dataset which are randomly distributed. <br>
Here, I take advantage of the size of the dataset to make a balanced sample i.e., there will be about the same number of `Fully Paid` and `Charged Off` applicants. This is done by defining a new column in the dataframe, named `loans['weights']` which determines the weights of each category in the `loan_status` column. In the weights column, all the weights except for `Fully Paid` and `Charged Off` are set to 0 i.e., the sample data will not have other categories. The weights of the two remaining columns are set to 1:5 to get a balance distribution of the two categories.

In [4]:
loans['weights'] = loans['loan_status'].map({'Fully Paid': 1, 'Charged Off': 5, 'Current': 0,'Late (31-120 days)':0,\
                                              'In Grace Period': 0,'Late (16-30 days)':0,'Issued': 0,\
                                               'Does not meet the credit policy. Status:Fully Paid':0,\
                                              'Does not meet the credit policy. Status:Charged Off': 0,'Default': 0})


In [5]:
loans_sample = loans.sample(frac = 0.1, weights='weights')
loans_sample['loan_status'].value_counts(normalize=True)

Fully Paid     0.50008
Charged Off    0.49992
Name: loan_status, dtype: float64

As seen, we have a balanced dataset in the sense that there are almost equal number of `Fully Paid` and `Charged Off` applicants

In [6]:
loans_sample['loan_status'] = loans_sample.loan_status.map({'Fully Paid':0, 'Charged Off':1})

In [7]:
#Drop the id column and weight column which was introduced earlier
loans_sample.drop(columns=['weights','id'], inplace=True)

In [8]:
loans_sample.shape

(292549, 140)

In [300]:
loan_sample = loans_sample.copy()

In [301]:
loans_sample.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
37956,10000.0,10000.0,10000.0,36 months,17.19%,357.48,C,C5,nails tech,5 years,MORTGAGE,48000.0,Source Verified,Jun-2019,1,n,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,Home improvement,157xx,PA,7.25,0.0,Oct-2014,675.0,679.0,0.0,NaN,NaN,8.0,0.0,12329.0,38.6%,13.0,1,0.0,0.0,1453.200000,1453.20,214.23,403.23,35.74,800.00,144.0000,Dec-2019,305.27,NaN,May-2020,604.0,600.0,0.0,NaN,1.0,0,NaN,NaN,NaN,0.0,0.0,12329.0,0.0,0.0,0.0,0.0,48.0,0.0,NaN,4.0,6.0,5761.0,39.0,31900.0,1.0,0.0,1.0,6.0,1541.0,5971.0,67.4,0.0,0.0,48.0,55.0,7.0,7.0,0.0,7.0,NaN,7.0,NaN,0.0,6.0,6.0,6.0,8.0,1.0,8.0,12.0,6.0,8.0,0.0,0.0,0.0,4.0,100.0,50.0,0.0,0.0,31900.0,12329.0,18300.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
27423,14000.0,14000.0,14000.0,60 months,14.99%,332.99,C,C4,Training Specialist,10+ years,RENT,70000.0,Source Verified,Mar-2017,1,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,113xx,NY,12.00,3.0,May-2001,670.0,674.0,0.0,13.0,NaN,9.0,0.0,7352.0,41.1%,22.0,1,0.0,0.0,11211.500000,11211.50,5286.99,4669.27,33.30,1221.94,219.9492,Dec-2019,365.30,NaN,Mar-2020,579.0,575.0,0.0,46.0,1.0,0,NaN,NaN,NaN,0.0,0.0,36937.0,1.0,1.0,1.0,1.0,6.0,29585.0,96.0,2.0,6.0,2146.0,76.0,17900.0,0.0,0.0,3.0,7.0,4617.0,3261.0,48.2,0.0,0.0,92.0,190.0,7.0,6.0,0.0,7.0,13.0,6.0,13.0,1.0,3.0,4.0,4.0,9.0,2.0,8.0,20.0,4.0,9.0,0.0,0.0,0.0,3.0,72.7,25.0,0.0,0.0,48647.0,36937.0,6300.0,30747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
372704,12800.0,12800.0,12800.0,60 months,9.99%,271.90,B,B3,Ultrasound tech,3 years,MORTGAGE,60000.0,Source Verified,Feb-2015,0,n,https://lendingclub.com/browse/loanDetail.a

In [302]:
loan_sample = loan_sample.reset_index(drop=True)

In [303]:
loan_sample.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,10000.0,10000.0,10000.0,36 months,17.19%,357.48,C,C5,nails tech,5 years,MORTGAGE,48000.0,Source Verified,Jun-2019,1,n,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,Home improvement,157xx,PA,7.25,0.0,Oct-2014,675.0,679.0,0.0,NaN,NaN,8.0,0.0,12329.0,38.6%,13.0,1,0.0,0.0,1453.200000,1453.20,214.23,403.23,35.74,800.00,144.0000,Dec-2019,305.27,NaN,May-2020,604.0,600.0,0.0,NaN,1.0,0,NaN,NaN,NaN,0.0,0.0,12329.0,0.0,0.0,0.0,0.0,48.0,0.0,NaN,4.0,6.0,5761.0,39.0,31900.0,1.0,0.0,1.0,6.0,1541.0,5971.0,67.4,0.0,0.0,48.0,55.0,7.0,7.0,0.0,7.0,NaN,7.0,NaN,0.0,6.0,6.0,6.0,8.0,1.0,8.0,12.0,6.0,8.0,0.0,0.0,0.0,4.0,100.0,50.0,0.0,0.0,31900.0,12329.0,18300.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,14000.0,14000.0,14000.0,60 months,14.99%,332.99,C,C4,Training Specialist,10+ years,RENT,70000.0,Source Verified,Mar-2017,1,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,113xx,NY,12.00,3.0,May-2001,670.0,674.0,0.0,13.0,NaN,9.0,0.0,7352.0,41.1%,22.0,1,0.0,0.0,11211.500000,11211.50,5286.99,4669.27,33.30,1221.94,219.9492,Dec-2019,365.30,NaN,Mar-2020,579.0,575.0,0.0,46.0,1.0,0,NaN,NaN,NaN,0.0,0.0,36937.0,1.0,1.0,1.0,1.0,6.0,29585.0,96.0,2.0,6.0,2146.0,76.0,17900.0,0.0,0.0,3.0,7.0,4617.0,3261.0,48.2,0.0,0.0,92.0,190.0,7.0,6.0,0.0,7.0,13.0,6.0,13.0,1.0,3.0,4.0,4.0,9.0,2.0,8.0,20.0,4.0,9.0,0.0,0.0,0.0,3.0,72.7,25.0,0.0,0.0,48647.0,36937.0,6300.0,30747.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,12800.0,12800.0,12800.0,60 months,9.99%,271.90,B,B3,Ultrasound tech,3 years,MORTGAGE,60000.0,Source Verified,Feb-2015,0,n,https://lendingclub.com/browse/loanDetail.acti...,debt_c

## Data Cleaning

At a glance, it is seen that the data set has a lot of Nan values, specially in some certain columns. We will drop the columns which have more than 10% of their data missing

### Dropping NaN Columns

In [304]:
loan_sample.isna().sum()

loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
                                               ...  
hardship_loan_status                          289029
orig_projected_additional_accrued_interest    288528
hardship_payoff_balance_amount                287417
hardship_last_payment_amount                  287417
debt_settlement_flag                               0
Length: 140, dtype: int64

In [305]:
# Finding all columns with more than 10% values missing
nan_cols = [i for i in loan_sample.columns if loan_sample[i].isnull().sum() > 0.1*len(loan_sample)]

In [306]:
# Dropping nan columns 
loan_sample.drop(nan_cols , axis=1, inplace=True)
loan_sample.shape

(292549, 90)

### Data Dictionary

In [307]:
data_Dic = pd.read_excel('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\LoanDataDictionary.xlsx')

In [308]:
data_Dic.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [309]:
# Joining the dictionary and column names
desc_loans_colms = pd.merge(pd.DataFrame({"col_name": list(loan_sample.columns)}), data_Dic, \
               how='inner', right_on='LoanStatNew', left_on='col_name')
desc_loans_colms.drop(['col_name'], axis =1, inplace=True)

In [310]:
# Displaying the dictionary defining each column
from IPython.display import display
with pd.option_context('display.max_rows', 100, 'display.max_columns', 3 , 'display.max_colwidth' , -1):
    display(desc_loans_colms)

,LoanStatNew,Description
0,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
1,funded_amnt,The total amount committed to that loan at that point in time.
2,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.
3,term,The number of payments on the loan. Values are in months and can be either 36 or 60.
4,int_rate,Interest Rate on the loan
5,installment,The monthly payment owed by the borrower if the loan originates.
6,grade,LC assigned loan grade
7,sub_grade,LC assigned loan subgrade
8,emp_title,The job title supplied by the Borrower when applying for the loan.*
9,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.


In [311]:
loan_sample.columns.shape

(90,)

* There are 90 columns in the dataset. However, looking at the data dictionary, one sees only 89 columns. The description of the column `total_rev_hi_lim` is missing in the Data Dictionary

Some of the features are related to a loan *after* the loan was approaved. As we are trying to predict a `risky` applicant based on their feature *before* they are approved, any feature related to payment of a loan and/or charging off a loan should be dropped. 

In [312]:
loan_sample.drop(columns = ['last_pymnt_d' , 'last_pymnt_amnt' , 'recoveries' , 'collection_recovery_fee' \
                            , 'debt_settlement_flag' , 'total_pymnt' , 'total_pymnt_inv' , 'total_rec_prncp'\
                           , 'total_rec_int' , 'total_rec_late_fee' , 'pymnt_plan' , 'last_credit_pull_d'] , inplace=True)

**Note**: The two columns `fico_range_high` and `fico_range_low` are very similar. In fact, the difference between the two columns in most of the cases is equal to 4 and for very rare cases it is equal to 5. 

In [313]:
(loan_sample['fico_range_high']-loan_sample['fico_range_low']).value_counts(normalize=True)

4.0    0.99986
5.0    0.00014
dtype: float64

We will take the average of the two column and replace them by their average.

In [314]:
# Replacing fico_range_low and fico_range_high by their average
loan_sample['fico_avg'] = (loan_sample['fico_range_high']+loan_sample['fico_range_low'])/2
loan_sample.drop(columns=['fico_range_high' , 'fico_range_low'] , inplace=True)

### Dropping NaN Rows

In [315]:
loan_sample.dropna(axis=0, inplace=True)

### Converting Non-numeric Columns to Numeric Columns

There are some columns which are listed as object but by a careful examination, they can be converted to numeric columns. Lets take a look at all the object columns

In [316]:
pd.set_option('display.max_columns', None)
loan_sample.select_dtypes(include='object')

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,url,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,hardship_flag
0,36 months,17.19%,C,C5,nails tech,5 years,MORTGAGE,Source Verified,Jun-2019,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,Home improvement,157xx,PA,Oct-2014,38.6%,N
1,60 months,14.99%,C,C4,Training Specialist,10+ years,RENT,Source Verified,Mar-2017,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,113xx,NY,May-2001,41.1%,N
2,60 months,9.99%,B,B3,Ultrasound tech,3 years,MORTGAGE,Source Verified,Feb-2015,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,754xx,TX,Jan-2005,30.5%,N
3,36 months,26.77%,G,G1,Submarine Technician,3 years,OWN,Verified,Aug-2015,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,967xx,HI,Feb-2011,60.6%,N
4,60 months,26.30%,E,E5,Journeyman electrician,2 years,MORTGAGE,Verified,Feb-2018,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,386xx,MS,Sep-2011,90.8%,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292543,60 months,12.69%,C,C2,Patient Coordinator,1 year,MORTGAGE,Verified,Jul-2015,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,774xx,TX,Feb-2003,31.2%,N
292545,60 months,18.06%,D,D2,Mechanic,3 years,RENT,Verified,Jul-2017,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,080xx,NJ,Apr-2011,43.5%,N
292546,60 months,16.99%,D,D3,Communications Manager,2 years,MORTGAGE,Source Verified,Sep-2015,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,330xx,FL,Oct-2004,46.5%,N
292547,36 months,8.39%,A,A5,welder,10+ years,MORTGAGE,Source Verified,Aug-2014,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,770xx,TX,Mar-1996,93.7%,N


Columns to be changed to numerics are: `term`, `int_rate`, `emp_length`, `revol_util`

In [317]:
# term
loan_sample.replace(to_replace=['36 months' , '60 months'], value=[36 , 60], regex=True , inplace = True)

In [318]:
# The int_rate column in not numeric. 
# Removing the % sign and converting int_rate col to a numceric col
loan_sample['int_rate']=loan_sample['int_rate'].str.replace('%', '').astype(float)

In [319]:
# The int_rate column in not numeric. 
# Removing the % sign and converting int_rate col to a numceric col
loan_sample['revol_util']=loan_sample['revol_util'].str.replace('%', '').astype(float)

In [320]:
# Emp_length
loan_sample['emp_length'].replace(to_replace=['years' , 'year' , '<' , '\+'], value='', regex=True, inplace = True)
loan_sample['emp_length']=loan_sample['emp_length'].astype(int)

In the numeric emp_length column, the 10+ years is changed to 10 and <1 year is changed to 1 but I will keep in mind that 10 represents 10+ and 1 represents 1 year or less of emp_length.

In [321]:
loan_sample['issue_year'] = pd.to_datetime(loan_sample['issue_d']).dt.year
loan_sample.drop(columns = 'issue_d' , inplace=True)

In [322]:
loan_sample['earliest_cr_line_year'] = pd.to_datetime(loan_sample['earliest_cr_line']).dt.year
loan_sample.drop(columns = 'earliest_cr_line' , inplace=True)

In [323]:
loan_sample['initial_list_status'].value_counts()

1    155590
0     78783
Name: initial_list_status, dtype: int64

In [324]:
loan_sample['initial_list_status'] = loan_sample['initial_list_status'].map({'w': 1, 'f': 0})

In [325]:
loan_sample['application_type'].value_counts()

0    225184
1      9189
Name: application_type, dtype: int64

In [326]:
loan_sample['application_type'] = loan_sample['application_type'].map({'Joint App': 1, 'Individual': 0})

In [327]:
loan_sample['hardship_flag'].value_counts()

N    234197
Y       176
Name: hardship_flag, dtype: int64

In [328]:
loan_sample['hardship_flag'] = loan_sample['hardship_flag'].map({'Y': 1, 'N': 0})

The `url` and `zid_code` are also not usefull features because they do not provide us with any information. We just drop them.

In [329]:
loan_sample.drop(columns=['url' , 'zip_code'] , inplace = True)

In [330]:
loan_sample.shape[1]

75

In [331]:
loan_sample.select_dtypes(include='object')

,grade,sub_grade,emp_title,home_ownership,verification_status,purpose,title,addr_state
0,C,C5,nails tech,MORTGAGE,Source Verified,home_improvement,Home improvement,PA
1,C,C4,Training Specialist,RENT,Source Verified,credit_card,Credit card refinancing,NY
2,B,B3,Ultrasound tech,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation,TX
3,G,G1,Submarine Technician,OWN,Verified,major_purchase,Major purchase,HI
4,E,E5,Journeyman electrician,MORTGAGE,Verified,debt_consolidation,Debt consolidation,MS
...,...,...,...,...,...,...,...,...
292543,C,C2,Patient Coordinator,MORTGAGE,Verified,debt_consolidation,Debt consolidation,TX
292545,D,D2,Mechanic,RENT,Verified,debt_consolidation,Debt consolidation,NJ
292546,D,D3,Communications Manager,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation,FL
292547,A,A5,welder,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation,TX


### Feature Selection

The importance of the numerical features is determined based on their correlation with the target column i.e., loan_status column.

In [336]:
corr_features = np.abs(loan_sample.corr()['loan_status'].drop('loan_status')).sort_values(ascending = False)
corr_features

last_fico_range_high    0.756774
last_fico_range_low     0.554640
int_rate                0.313780
term                    0.209836
fico_avg                0.163967
                          ...   
initial_list_status          NaN
out_prncp                    NaN
out_prncp_inv                NaN
policy_code                  NaN
application_type             NaN
Name: loan_status, Length: 66, dtype: float64

We can introduce a threshold for minimum of a correction a variable should have in order to keep it for modeling. Here, I choose the threshold to be 0.02 i.e., any feature that has a correlation less than 0.02 will be dropped.

In [337]:
important_numeric_features = corr_features[corr_features>0.02].index
list(important_numeric_features)

['last_fico_range_high',
 'last_fico_range_low',
 'int_rate',
 'term',
 'fico_avg',
 'acc_open_past_24mths',
 'num_tl_op_past_12m',
 'bc_open_to_buy',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'funded_amnt',
 'loan_amnt',
 'funded_amnt_inv',
 'mort_acc',
 'dti',
 'total_bc_limit',
 'tot_cur_bal',
 'mo_sin_old_rev_tl_op',
 'installment',
 'num_rev_tl_bal_gt_0',
 'num_actv_rev_tl',
 'inq_last_6mths',
 'total_rev_hi_lim',
 'mo_sin_rcnt_tl',
 'mo_sin_rcnt_rev_tl_op',
 'mths_since_recent_bc',
 'earliest_cr_line_year',
 'bc_util',
 'percent_bc_gt_75',
 'revol_util',
 'num_actv_bc_tl',
 'num_op_rev_tl',
 'mo_sin_old_il_acct',
 'num_sats',
 'open_acc',
 'pub_rec_bankruptcies',
 'revol_bal',
 'issue_year',
 'pub_rec',
 'emp_length',
 'hardship_flag']

The difference between fico_range_low and fico_range_high is 4 for 99% of the sample data (shown below). Later on in this notebook, a new column called fico_avg will be created as fico_avg=(fico_range_low+fico_range_high)/2

In [338]:
important_category_features = list(loan_sample.select_dtypes(['object']).columns)
important_category_features

['grade',
 'sub_grade',
 'emp_title',
 'home_ownership',
 'verification_status',
 'purpose',
 'title',
 'addr_state']

We can now combine the important numeric and categorical features to get a list of important features.

In [339]:
important_features = [*important_numeric_features , *important_category_features]
important_features

['last_fico_range_high',
 'last_fico_range_low',
 'int_rate',
 'term',
 'fico_avg',
 'acc_open_past_24mths',
 'num_tl_op_past_12m',
 'bc_open_to_buy',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'funded_amnt',
 'loan_amnt',
 'funded_amnt_inv',
 'mort_acc',
 'dti',
 'total_bc_limit',
 'tot_cur_bal',
 'mo_sin_old_rev_tl_op',
 'installment',
 'num_rev_tl_bal_gt_0',
 'num_actv_rev_tl',
 'inq_last_6mths',
 'total_rev_hi_lim',
 'mo_sin_rcnt_tl',
 'mo_sin_rcnt_rev_tl_op',
 'mths_since_recent_bc',
 'earliest_cr_line_year',
 'bc_util',
 'percent_bc_gt_75',
 'revol_util',
 'num_actv_bc_tl',
 'num_op_rev_tl',
 'mo_sin_old_il_acct',
 'num_sats',
 'open_acc',
 'pub_rec_bankruptcies',
 'revol_bal',
 'issue_year',
 'pub_rec',
 'emp_length',
 'hardship_flag',
 'grade',
 'sub_grade',
 'emp_title',
 'home_ownership',
 'verification_status',
 'purpose',
 'title',
 'addr_state']

In [342]:
loan_sample = loan_sample[important_features]

In [ ]:
#Saving the sample data
loan_sample.to_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\loan_sample.csv' , index=False)

In [ ]:
#loan_sample = pd.read_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\loan_sample.csv')

# References
[1] https://en.wikipedia.org/wiki/LendingClub